### **Content License Agreement**

<font color='red'><b>**WARNING**</b></font> : 본 자료는 삼성 청년 SW아카데미의 컨텐츠 자산으로, 보안서약서에 의거하여 어떠한 사유로도 임의로 복사, 촬영, 녹음, 복제, 보관, 전송하거나 허가 받지 않은 저장매체를 이용한 보관, 제3자에게 누설, 공개 또는 사용하는 등의 무단 사용 및 불법 배포 시 법적 조치를 받을 수 있습니다.

### **Objectives**

1. 실습명 : 합성 데이터 제작
2. 핵심 주제:
    1. 환경 설정 및 기본 함수 이해
    2. 데이터 '생성'을 위한 Prompt Engineering
    3. 합성 데이터 평가
3. 학습 목표 :
    1. 과제 진행에 필요한 라이브러리와 API 정보를 설정하고, 기본 통신 함수를 이해(httpx, json 라이브러리 임포트)
    2. 가상의 '영화 추천 챗봇'을 위한 학습 데이터를 생성
    3. 생성된 데이터의 품질을 자동으로 평가하는 '평가자(Judge)' LLM을 설계

4. 학습 개념: 키워드명 :
    1. API 통신
    2. Prompt Engineering
    3. LLM as Judge
  
5. 학습 방향 :
  - 실습은 기본적인 API 통신을 통해 데이터를 생성하고, 합성 데이터의 품질을 평가하는 '평가자(Judge)' LLM을 설계하는 과정을 통해 학습됩니다.
  - 실습 코드는 조교가 직접 구현한 코드를 참고하며 학습합니다.
  - 해당 실습에 정답은 없으므로 자신이 옳다고 생각하는 나름의 근거를 세워가며 학습합니다.



### **Prerequisites**


In [ ]:
%pip install dotenv==0.9.9 openai==1.100.0

# 1. 환경설정 및 관련 라이브러리 설명

- os environment variable에 대해서 알아봅니다.
- httpx에 대해서 알아봅니다.
- json에 대해서 알아봅니다.


### 1.1. os environment variable

🧾 환경 변수(Environment Variable)란?

환경 변수(Environment Variable)는 운영체제(Windows, macOS, Linux 등)가 프로그램 실행 환경에 제공하는 전역 설정값입니다.
프로그램은 이 환경 변수를 통해 중요한 정보(예: API 키, DB 비밀번호, 시스템 경로)를 읽을 수 있습니다.

<blockquote>
<b>🧠 왜 환경 변수를 쓸까요?</b><br>
환경 변수는 코드 외부에서 안전하게 관리할 수 있습니다.
</blockquote>

1. 보안
	•	코드 안에 직접 비밀번호나 API Key를 적으면 위험합니다.
→ GitHub에 올리거나 다른 사람에게 코드 파일을 공유하면 키가 유출될 수 있습니다.
	•	환경 변수는 운영체제 메모리에만 저장되므로, 코드 외부에서 안전하게 관리할 수 있습니다.

2. 유연성
	•	같은 코드라도, 환경 변수만 바꾸면 다른 API 키나 설정을 사용할 수 있습니다.
	•	예를 들어, 개발 서버와 배포 서버에서 다른 API 키를 사용하려면 코드 수정 없이 환경 변수만 변경하면 됩니다.

그러면 환경변수를 .env 파일에 저장하고, os.environ을 사용하여 환경변수를 읽는 방법을 사용해보도록 하겠습니다.

구글 드라이브에 mount를 합니다.

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")

Mounted at /content/drive


mount한 이후 아래 코드를 실행하여 .env 파일을 생성합니다.

API key는 [Upstage Console](https://console.upstage.ai/docs/getting-started)을 확인해주세요.

In [ ]:
# your_api_key = "<YOUR_API_KEY>" # 여기에 Upstage API 키를 입력하세요.

# # .env 파일에 쓰기
# !echo "UPSTAGE_API_KEY={your_api_key}" > "/content/drive/My Drive/Colab Notebooks/.env"

저장된 .env 파일을 불러서 환경변수로 설정합니다.

In [ ]:
# from dotenv import load_dotenv
# from os import getenv
# load_dotenv("/content/drive/My Drive/Colab Notebooks/.env")

# UPSTAGE_API_KEY = getenv("UPSTAGE_API_KEY")
# if UPSTAGE_API_KEY:
#     print("Success API Key Setting!")

Success API Key Setting!


이렇게 불러오면 .env 파일을 github에 push할 때, gitignore 처리를 해서 API가 유출되지 않도록 할 수 있습니다.

또한, .env 파일을 암호화해서 보관할 수도 있습니다.

### 1.2. httpx
httpx는 Python용 차세대 HTTP 클라이언트 라이브러리로,
유명한 requests 라이브러리의 API 스타일을 계승하면서도 **비동기(Async)**와 HTTP/2를 지원하도록 설계되었습니다.
- Python 3.6 이상에서 동작하며, 동기·비동기 모두 지원하는 것이 가장 큰 특징입니다.
- timeout 옵션을 세밀하게 제어 가능합니다.

<blockquote>
<b>🧠 httpx를 왜 쓸까요?</b><br>
httpx를 사용하는 이유는 <strong>비동기 통신</strong>을 하기 위해서 입니다.
</blockquote>

📌 비동기 통신을 하게 되면 어떤 장점이 있을까요?

1. 네트워크 요청을 기다리는 동안 프로그램이 멈추지 않고 다른 작업을 병렬로 처리할 수 있습니다.
2. 동기로 API를 호출하면 해당 응답을 받을 때까지 기다려야 합니다.
3. python은 기본적으로 동기적으로 동작하기 때문에 이는 매우 비효율적입니다.
4. 비동기 API 호출을 하게 되면 코드상에서 한번에 여러 API를 호출할 수 있습니다.

이러한 이유로 비동기 통신을 사용하는 것이 매우 중요합니다. 그러면 아래 코드로 비동기 통신을 구현해보겠습니다.

In [2]:
import httpx

async def call_chat_completion(url: str, headers: dict, payload: dict):
    async with httpx.AsyncClient(timeout=30.0) as client:
        response = await client.post(url, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()

        return data["choices"][0]["message"]["content"]


위에서 처럼 `async with httpx.AsyncClient() as client:` 을 사용하여 REST API방식으로 비동기적으로 호출을 할 수 있습니다.

그러면 실제로 파라미터 값들을 넣고 간단한 호출을 진행해보겠습니다.

In [5]:
import asyncio
from google.colab import userdata


async def main():
    url = "https://api.upstage.ai/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {userdata.get('UPSTAGE_API_KEY')}"
    }
    prompts = [
        "Tell me a joke about cats",
        "What is the capital of France?",
        "Summarize the plot of Inception in 2 sentences."
    ]

    tasks = []
    for prompt in prompts:
        payload = {
            "model": "solar-pro2",
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "stream": False
        }
        tasks.append(call_chat_completion(url, headers, payload))

    results = await asyncio.gather(*tasks)

    for i, res in enumerate(results, 1):
        print(f"--- Response {i} ---")
        print(res)
        print()

# 🔹 실행
await main()

--- Response 1 ---
Why don't cats play poker in the jungle?  
Because there are too many *cheetahs*!  

😸 (Get it? *Cheat*-ahs? Okay, I'll show myself out.)  

Want another one? They're *purr*-fectly bad! 😸

--- Response 2 ---
The capital of France is **Paris**.  

Known for its iconic landmarks like the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral, Paris is one of the world's most visited cities and a major cultural, political, and economic center.  

Let me know if you'd like more fun facts about Paris! 😊

--- Response 3 ---
In *Inception*, a thief named Dom Cobb is offered the chance to have his criminal history erased as payment for a seemingly impossible task: "inception," the implantation of an idea into the mind of a C.E.O., using shared dreaming technology that allows entry into the dreams of others. The mission goes awry when the team confronts unforeseen dangers in layered dreamscapes, and Cobb must confront personal ghosts from his past to return safely to his child

3개의 응답을 한번에 받을 수 있었습니다.

위의 코드는 아래 python 코드와 동일한 결과를 얻을 수 있습니다.

In [6]:
from openai import AsyncOpenAI

client = AsyncOpenAI(api_key=userdata.get('UPSTAGE_API_KEY'), base_url="https://api.upstage.ai/v1")

async def chat_completion(prompt: str, model: str = "solar-pro2") -> str:
    """
    비(非)스트리밍 호출 버전. 한 번에 전체 응답을 받아옵니다.
    """
    resp = await client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        stream=False,
    )
    return resp.choices[0].message.content

async def main():
    prompts = [
        "Tell me a joke about cats",
        "What is the capital of France?",
        "Summarize the plot of Inception in 2 sentences."
    ]

    # ✅ 동시에 스트리밍 호출
    tasks = [chat_completion(p) for p in prompts]
    results = await asyncio.gather(*tasks)

    # 결과 정리 출력
    for i, res in enumerate(results, 1):
        print(f"--- Response {i} (collected) ---")
        print(res)
        print()

# Colab/Jupyter에서는 최상단 셀에서 바로 실행 가능
await main()

--- Response 1 (collected) ---
Sure! Here's a purr-fect one for you:

Why don't cats play poker in the jungle?

Because there are too many cheetahs!  

😸 *Meow-velous, right?* 😸

--- Response 2 (collected) ---
The capital of France is **Paris**.  

Would you like any other facts about Paris or France? 😊

--- Response 3 (collected) ---
In *Inception*, a skilled thief named Dom Cobb is offered the chance to have his criminal history erased as payment for a seemingly impossible task: "inception," the implantation of an idea into the mind of a C.E.O., using shared dreaming technology that allows entry into the depths of the human psyche. The mission goes awry as layers of dreams within dreams blur reality, forcing Cobb to confront his unresolved past and the dangers of his own subconscious.



### 1.3. JSON

<blockquote>
<b>🧠 JSON이란?</b><br>
JSON은 JavaScript Object Notation의 약자입니다.
</blockquote>

데이터를 저장하거나 주고받을 때 쓰는 가볍고 읽기 쉬운 데이터 형식입니다.
이름에 JavaScript가 들어있지만, Python, Java, C#, Go, Swift 등 대부분의 프로그래밍 언어에서 모두 사용할 수 있습니다.

1. 왜 JSON을 쓰나요?
	•	사람이 읽기 쉽다 → {}와 [], "키": 값 형태로 구조가 명확합니다.
	•	컴퓨터가 처리하기 쉽다 → 모든 언어에서 JSON을 쉽게 읽고 쓸 수 있는 라이브러리가 있음.
	•	언어에 독립적 → 어떤 언어에서 만든 JSON도 다른 언어에서 그대로 읽을 수 있음.
	•	웹 표준 → API, 서버·클라이언트 데이터 통신에 가장 널리 쓰임.

2. JSON의 기본 문법
객체(Object) : {} 안에 "키": 값 쌍으로 구성되어 있습니다.
```json
{
  "name": "Alice",
  "age": 25,
  "is_student": false
}
```
3. 모델의 structured output에서는 보통 JSON의 형식을 많이 사용합니다.
- LLM 모델은 지정된 형식을 따라 generate할 수 있도록 만들 수 있는데, 이때 대표적으로 사용하는 형식이 JSON 형식입니다.

아래 코드를 실행하여 어떤 방식으로 JSON 형식으로 만드는지 확인해봅니다.

In [7]:
from openai import OpenAI
import json

client = OpenAI(
    api_key=userdata.get('UPSTAGE_API_KEY'),
    base_url="https://api.upstage.ai/v1"
)

user_prompt = """가상의 데이터를 만들려고 합니다.

아래 output_format에 따라 JSON 데이터를 1개만 반환해주세요.
{output_format}
"""

# TODO: output_format을 정의해주세요.
output_format ="""<output_format>
```json
{
    "name": [이름],
    "age": [나이]
    "is_student": [true 또는 false]
}
```
</output_format>"""

messages=[
    {
        "role": "user",
        "content": user_prompt.format(output_format=output_format)
    }
]

response = client.chat.completions.create(
    model="solar-pro2",
    messages=messages,
)

def json_parsing(output_text:str) -> dict:
    output_text = output_text[output_text.index("```json") + len("```json"):].strip()
    output_text = output_text[:output_text.index("```")].strip()
    return json.loads(output_text)


output = response.choices[0].message.content
print("현재 응답의 형식: ", type(output))
print(output)
structured_dictionary = json_parsing(output)
print("구조화된 응답의 형식: ", type(structured_dictionary))
print(structured_dictionary)

현재 응답의 형식:  <class 'str'>
```json
{
    "name": "김지은",
    "age": 24,
    "is_student": false
}
```
구조화된 응답의 형식:  <class 'dict'>
{'name': '김지은', 'age': 24, 'is_student': False}


<blockquote>
<b>🧠 response_format이란?</b><br>
prompt에 json 형식의 output을 정의하지 않고 response_format에 직접 json schema를 정의해서 output format을 정의할 수도 있습니다.
</blockquote>

1. 프롬프트에 어떤 output format을 지정하게 되면 응답 결과물에 추가로 parsing 함수를 정의해서 json으로 변환 가능한 텍스트만 추출해야 합니다.
2. 만약, 모델의 성능이 떨어지는 상횡이 생기면 parsing이 안될 가능성도 존재합니다.
3. 따라서, 좀더 안정적인 결과물을 얻기 위해서는 `response_format`을 사용하는 것이 좋습니다.

자세한 사용법은 [openai structured output](https://platform.openai.com/docs/guides/structured-outputs?lang=python)에 대한 문서를 참고해주세요.


In [8]:
from openai import OpenAI
import json


messages=[
    {
        "role": "system",
        "content": "You are an expert in information extraction. Extract information from the given HTML representation of image and organize them into a clear and accurate JSON format."
    },
    {
        "role": "user",
        "content": "HTML string: <table id='0' style='font-size:14px'><tr><td>1</td><td>FUTAMI 17 GREEN TEA (CLAS</td><td>12,500</td></tr><tr><td>1</td><td>EGG TART</td><td>13,000</td></tr><tr><td>1</td><td>GRAIN CROQUE MONSIEUR</td><td>17,000</td></tr></table><br><table id='1' style='font-size:18px'><tr><td>TOTAL</td><td>42, 500</td></tr><tr><td>CASH</td><td>50,000</td></tr><tr><td></td><td></td></tr><tr><td>CHANGE</td><td>7 ,500</td></tr></table>\n. Extract the structured data from the HTML string in JSON format."
    }
]

response_format={
    "type": "json_schema",
    "json_schema": {
        "name": "restaurant_receipt",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "menu_items": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "menu_cnt": {
                                "type": "number",
                                "description": "The count of the menu item."
                            },
                            "menu_name": {
                                "type": "string",
                                "description": "The name of the menu item."
                            },
                            "menu_price": {
                                "type": "number",
                                "description": "The price of the menu item."
                            }
                        },
                        "required": ["menu_cnt", "menu_name", "menu_price"],
                    }
                },
                "total_price": {
                    "type": "number",
                    "description": "The total price of the receipt."
                }
            },
            "required": ["menu_items", "total_price"],
        }
    }
}

response = client.chat.completions.create(
    model="solar-pro2",
    messages=messages,
    response_format=response_format
)

structured_output = response.choices[0].message.content
print("현재 응답의 형식: ", type(structured_output))
print(structured_output)
structured_dictionary = json.loads(structured_output)
print("구조화된 응답의 형식: ", type(structured_dictionary))
print(structured_dictionary)

현재 응답의 형식:  <class 'str'>
{
  "menu_items": [
    {
      "menu_cnt": 1,
      "menu_name": "FUTAMI 17 GREEN TEA (CLAS",
      "menu_price": 12500
    },
    {
      "menu_cnt": 1,
      "menu_name": "EGG TART",
      "menu_price": 13000
    },
    {
      "menu_cnt": 1,
      "menu_name": "GRAIN CROQUE MONSIEUR",
      "menu_price": 17000
    }
  ],
  "total_price": 42500
}
구조화된 응답의 형식:  <class 'dict'>
{'menu_items': [{'menu_cnt': 1, 'menu_name': 'FUTAMI 17 GREEN TEA (CLAS', 'menu_price': 12500}, {'menu_cnt': 1, 'menu_name': 'EGG TART', 'menu_price': 13000}, {'menu_cnt': 1, 'menu_name': 'GRAIN CROQUE MONSIEUR', 'menu_price': 17000}], 'total_price': 42500}


확인하신 것처럼 JSON으로 변환이 가능한 string 타입의 응답을 반환합니다. 해당 응답을 `json.loads()`를 사용하여 dictionary 타입으로 변환할 수 있습니다.

만약, 모델의 성능이 떨어지게 되면 JSON으로 변환이 불가능한 string 타입으로 반환할 수 있으므로 `try except문`을 통해 에러 처리를 해주시면 좋습니다.

# 2. 데이터 '생성'을 위한 Prompt Engineering

- 데이터 생성을 위한 Prompt Engineering에 대해 알아봅니다.
- 데이터 생성을 위한 API를 사용하여 데이터를 생성합니다.
- 생성된 데이터를 확인합니다.

### 2.1. 프롬프트 엔지니어링

<blockquote>
<b>🧠 프롬프트 엔지니어링(Prompt Engineering)이란?</b><br>
AI 모델에게 원하는 답을 정확하고 효율적으로 이끌어내기 위해 입력(프롬프트)을 설계·최적화하는 기술을 말합니다.
</blockquote>

1. 프롬프트 엔지니어링은 모델의 성능을 크게 좌우합니다.
	- AI 모델은 **질문(프롬프트)**에 따라 같은 데이터라도 완전히 다른 답을 내놓습니다.
		- 잘못된 프롬프트 → 모호하거나 엉뚱한 답변
		- 잘 설계된 프롬프트 → 정확하고 원하는 형식의 답변
	- 프롬프트 엔지니어링은 AI를 내 마음대로 컨트롤하는 리모컨과 같습니다.

2. 프롬프트의 기본 구조
하나의 프롬프트는 보통 다음 요소들로 구성됩니다.
	1. 역할(Role) 설정 → AI의 “정체성”과 “관점”을 지정합니다.
		- 예: “당신은 10년 경력의 여행 가이드입니다.”
	2. 목표(Task) 명확화 → 무엇을 해야 하는지 구체적으로 지시합니다.
		- 예: “서울에서 하루 동안 즐길 수 있는 코스를 추천해주세요.”
	3. 조건(Constraints) 부여 → 답변 길이, 형식, 언어, 스타일 등 제한사항을 명시합니다.
		- 예: “표 형식으로 3개의 추천 코스를 제시하고, 각 코스에 사진 링크를 포함하세요.”
	4.	예시(Examples) 제공 (Few-shot Prompting) → 원하는 답변 패턴을 보여주면 AI가 비슷하게 답변합니다.
		- 예: “예시: 코스 이름 - 설명 - 추천 이유”

3. 좋은 프롬프트 작성 원칙
	- 명확성 → 애매한 단어 대신 구체적인 지시
	- 맥락 제공 → 필요한 배경정보 포함
	- 출력 형식 지정 → 표, 목록, JSON 등
	- 예시 제시 → 원하는 결과물의 예시를 보여줌
	- 점진적 요청 → 복잡한 문제는 단계별로 쪼개서 요청

그러면 실제로 시스템 프롬프트를 다르게 작성했을때 어떻게 답변 퀄리티가 달라지는지 확인해보겠습니다.

In [9]:
GENERATOR_SYSTEM_PROMPT = """당신은 세상의 모든 영화를 꿰뚫고 있는 영화 전문가 '시네마스터'입니다.
사용자의 요청에 맞춰 영화를 추천하는 역할을 맡고 있습니다. 영화는 반드시 하나만 추천합니다.

{rule}

추천할 때는 반드시 영화 제목, 개봉 연도, 그리고 추천 이유를 포함해야 합니다.
"""

# TODO: 아래 두 가지 규칙 중 하나를 선택하여 지시사항을 완성하세요.
# 규칙 A (전문가 모드): 전문적인 용어를 사용하고, 영화의 숨겨진 의미나 감독의 의도를 함께 설명해주세요.
# 규칙 B (친구 모드): 친근하고 유머러스한 말투로, 왜 이 영화가 재미있는지 쉽게 설명해주세요.
rule = "규칙 B (친구 모드): 친근하고 유머러스한 말투로, 왜 이 영화가 재미있는지 쉽게 설명해주세요."

client = OpenAI(
    api_key=userdata.get('UPSTAGE_API_KEY'),
    base_url="https://api.upstage.ai/v1"
)

response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "system",
            "content": GENERATOR_SYSTEM_PROMPT.format(rule=rule)
        },
        {
            "role": "user",
            "content": "공포 영화를 추천해줘"
        }
    ],
)

output = response.choices[0].message.content
print(output)

**🎬 "헤어질 결심" (2022) 대신 진짜 공포물 추천! "허스탈" (2019)** 🎬  

"헤어질 결심"은 스릴러라 공포는 아니니까~ 대신 **"허스탈" (HEREDITARY, 2018)**을 강력 추천합니다!  

**✨ 추천 이유:**  
- **"가족"**이라는 평범한 설정에서 시작되는 **극악의 심리적 공포**! 악마적 분위기가 스크린을 압도합니다.  
- **충격적인 전개**와 **엄마(파멜라)의 기괴한 연기**가 소름을 돋게 하죠. (특히 초반 장면들은 평생 잊히지 않음)  
- **"펀 하우스"** 클라이맥스는 **"이게 실사 영화 맞나?"** 싶을 정도로 소름.  

**💀 주의:** 잠잘 때 침대 밑에 괴물이 있을 것 같은 불안감이 **1주일** 갑니다. (웃음)  
그래도 찐 공포를 원한다면 **"허스탈"이 정답!** 🎅👻


규칙에 따라 말투가 바뀌는 것을 확인하실 수 있습니다.

즉, 어떤 규칙을 설정하느냐에 따라서 답변의 퀄리티가 달라지는 것에 유의하시길 바랍니다.

### 2.2. 합성 데이터

<blockquote>
<b>🧠 합성 데이터(Synthetic Data)란?</b><br>
합성 데이터는 실제 세상에서 수집한 데이터가 아니라, 인공적으로 만들어낸 데이터입니다.
</blockquote>
합성 데이터는 아래와 같은 목적으로 사용될 수 있습니다.

1.  데이터 부족 문제 해결
	- 새로운 AI 모델을 만들 때, 필요한 데이터가 부족할 수 있습니다.
	- 예: 희귀 질병 진단, 특수 상황의 교통 데이터 등.

2.  비용 절감
	- 실제 데이터를 수집·가공하는 데는 많은 시간과 돈이 듭니다.
	- 합성 데이터는 컴퓨터로 빠르고 저렴하게 생성 가능.

3.  개인정보 보호
	- 의료 기록, 금융 거래처럼 민감한 정보는 직접 사용하기 어렵습니다.
	- 합성 데이터는 실제 사람의 개인정보를 포함하지 않으면서도, 통계적으로 비슷한 특성을 가질 수 있습니다.

4.  다양한 상황 테스트
	- 실제로는 드물게 발생하는 사건(예: 비행기 엔진 고장, 교통사고)을 AI가 학습할 수 있도록 미리 만들어 학습·테스트 가능.

합성데이터는 실제 데이터과 유사하게 만들어야 합니다. 그 중에서도 저희는 "영화 추천" 챗봇을 위한 학습 데이터를 생성하기 위해서 합성 데이터를 만들어보겠습니다.

아래 실행 원칙을 지켜가며 합성 데이터를 생성해보겠습니다.
1. 위의 시스템 프롬프트에 결과물의 일관성을 유지하기 위해 output_format을 설정합니다.
2. 합성 데이터를 무수히 만들더라도 비슷한 데이터만 많이 만들면 학습의 효율성이 떨어질 수 있습니다. 따라서, 다양성을 높여주는 `temperature` 값을 조절합니다. `temperature=0.0`이면 다양성이 매우 낮은 것을 의미합니다.
3. 중복되는 데이터 등 불필요한 데이터가 생길 수 있으므로 필요하다면 필터링 함수도 추가합니다.

In [10]:
# TODO: SYSTEM PROMPT를 작성하세요
STRUCTURED_GENERATOR_SYSTEM_PROMPT = """당신은 세상의 모든 영화를 꿰뚫고 있는 영화 전문가 '시네마스터'입니다.
사용자의 요청에 맞춰 영화를 추천하는 역할을 맡고 있습니다. 영화는 반드시 하나만 추천합니다.

## 1. 입력 형식
[추천받고자 하는 영화 장르]

## 2. 작업 지시
{rule}

## 3. 출력 형식
<output_format>
```json
{{
    "movie_name": [영화 이름],
    "year": [개봉 연도],
    "reason": [추천 이유]
}}
```
</output_format>
"""
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "system",
            "content": STRUCTURED_GENERATOR_SYSTEM_PROMPT.format(rule=rule)
        },
        {
            "role": "user",
            "content": "공포 영화를 추천해줘"
        }
    ],
    temperature=1.0,
)

output = response.choices[0].message.content
structured_dictionary = json_parsing(output)
print("구조화된 응답의 형식: ", type(structured_dictionary))
print(structured_dictionary)

구조화된 응답의 형식:  <class 'dict'>
{'movie_name': '헤어질 결심', 'year': 2022, 'reason': "와! '헤어질 결심'은 공포 분류되진 않았지만, 박찬욱 감독 특유의 숨 막히는 긴장감과 미스터리한 분위기가 진짜 소름 돋는 거예요. 로맨스와 스릴러가 뒤섞인 독특한 매력에 빠져들다가 결국 주인공이 당신을 붙잡고 놓지 않을 거야. '이게 공포 아니냐' 싶을 정도로 심장이 철렁철렁! 한국 영화의 새로운 장르 체험을 원한다면 완전 추천합니다~🔥"}


### 2.3. 합성 데이터 평가

<blockquote>
<b>🧠 LLM as a Judge란?</b><br>
사람 대신 LLM 모델을 이용하여 정성적인 결과물을 평가하는 것을 의미합니다.
</blockquote>

1. LLM as a Judge라는 말은 LLM이 판사처럼 다른 답변을 평가한다는 뜻입니다.
- 어떤 질문에 대해 여러 학생이 답을 했다고 해본다고 하는 경우에
- 그러면 LLM이 각 답을 읽고, “이 답은 정확한가?”, “논리가 맞는가?”, “설명이 충분한가?” 같은 기준으로 점수를 매깁니다.

2. 사용 이유
- 자동 채점: 시험 문제나 과제를 사람이 일일이 채점하는 대신, LLM이 먼저 평가를 도와줄 수 있습니다. (비용 효율적)
- 피드백 제공: 단순히 점수만 주는 게 아니라, 어떤 부분이 좋고 어떤 부분이 부족한지 설명까지 덧붙일 수 있습니다.

3. 장점과 한계
- 장점: 빠르고, 일관성 있게 많은 답을 평가할 수 있습니다.
- 한계: LLM도 완벽하지 않아서 때때로 잘못된 평가를 할 수 있습니다. 평가를 잘 하기 위한 다양한 원칙을 지켜야 합니다.

4. 사용 원칙
- 평가자 모델은 추론에 사용한 모델보다 더 큰 모델을 사용해야 합니다. 또한, 추론에 사용한 모델과 평가자 모델이 같을 경우 호의적인 평가를 내리는 bias가 발생하므로 다른 모델로 사용해야 합니다.
- 평가 결과물은 점수(score)와 이유(reason)로 구성됩니다. reason을 생성하게 함으로써 모델이 좀더 정확한 평가를 하게 만듭니다.
- 평가 결과물이 일관되도록 temperature를 0으로 설정합니다.

그러면 LLM as a Judge를 이용하여 이전에 생성한 합성 데이터 결과물을 평가해보겠습니다.

In [12]:
JUDGE_SYSTEM_PROMPT="""당신의 역할은 모델 답변 자동 평가자입니다. 입력 프롬프트와 모델 답변을 보고, 평가 기준에 따라 모델 답변을 평가합니다.

## 1. 입력 형식
    - 입력 프롬프트: [instruction]
    - 모델 답변: [output]
    - 평가 기준: [criteria]

## 2. 작업 지시
    - [instruction]에 따른 모델 결과물인 [output]을 평가합니다.
    - [output]은 [criteria]를 충족하는지 평가합니다.

## 3. 채점 원칙 (각 기준별 1–5점, 정수만)
    - 5점 (탁월): 기준을 완전히 충족. 오류·누락 없음. 구체적이고 실행가능.
	- 4점 (우수): 대체로 충족. 사소한 흠만 있음(정확성·구체성·형식 등에서 경미한 누락).
	- 3점 (보통): 핵심은 맞지만 눈에 띄는 약점 존재(누락, 모호함, 근거 부족 등).
    - 2점 (미흡): 중요한 요구를 여러 곳에서 놓침 또는 오류/비논리 다수.
	- 1점 (부적합): 전반적으로 요청과 어긋남, 의미있는 도움/근거 없음, 안전·정책 위반 가능성.

## 3. 출력 형식 (엄격 준수)
	- "score"는 1–5점의 정수로 평가한다.
	- "reason"는 한국어 1–3문장으로 평가한다. 구체적이고 실행 가능하게 작성한다.
	- 출력 형식은 JSON 형식인 <output_format>을 준수한다.

<output_format>
```json
{{
    "score": [모델의 답변 평가 점수],
    "comment": [평가 주석]
}}
```
"""

USER_PROMPT = """- 입력 프롬프트: {instruction}
- 모델 답변: {output}
- 평가 기준: {criteria}
"""

instruction = GENERATOR_SYSTEM_PROMPT.format(rule=rule)
# TODO: 기준을 정의해주세요
criteria = "요청의 충실도"

print(USER_PROMPT.format(instruction=instruction, output=output, criteria=criteria))

messages = [
        {
            "role": "system",
            "content": JUDGE_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": USER_PROMPT.format(instruction=instruction, output=output, criteria=criteria)
        }
]

response = client.chat.completions.create(
    model="solar-pro2",
    messages=messages
)

llm_as_judge_output = response.choices[0].message.content
llm_as_judge_structured_dictionary = json_parsing(llm_as_judge_output)
print("구조화된 응답의 형식: ", type(llm_as_judge_structured_dictionary))
print(llm_as_judge_structured_dictionary)

- 입력 프롬프트: 당신은 세상의 모든 영화를 꿰뚫고 있는 영화 전문가 '시네마스터'입니다.
사용자의 요청에 맞춰 영화를 추천하는 역할을 맡고 있습니다. 영화는 반드시 하나만 추천합니다.

규칙 B (친구 모드): 친근하고 유머러스한 말투로, 왜 이 영화가 재미있는지 쉽게 설명해주세요.

추천할 때는 반드시 영화 제목, 개봉 연도, 그리고 추천 이유를 포함해야 합니다.

- 모델 답변: <output_format>
```json
{
    "movie_name": "헤어질 결심",
    "year": 2022,
    "reason": "와! '헤어질 결심'은 공포 분류되진 않았지만, 박찬욱 감독 특유의 숨 막히는 긴장감과 미스터리한 분위기가 진짜 소름 돋는 거예요. 로맨스와 스릴러가 뒤섞인 독특한 매력에 빠져들다가 결국 주인공이 당신을 붙잡고 놓지 않을 거야. '이게 공포 아니냐' 싶을 정도로 심장이 철렁철렁! 한국 영화의 새로운 장르 체험을 원한다면 완전 추천합니다~🔥"
}
```
>

**아, 잠깐!** 혹시 전통 공포물을 원하셨다며? 그렇다면 **"곡성" (2016)**을 대신 추천해 드릴게요.  
악귀와 미신으로 뒤틀린 마을, 나홍진 감독의 **"진짜 한국식 공포"**가 무엇인지 보여주죠. 특히 할머니의 **"무서버~"** 대사는 아직도 귓가에 맴돈다는... ⚡  

(원하는 스타일이 있다면 언제든 말해주세요!)
- 평가 기준: 요청의 충실도

구조화된 응답의 형식:  <class 'dict'>
{'score': 4, 'comment': "규칙 B(친구 모드)를 반영해 유머와 친근감을 담아 설명했으며, 영화 제목·개봉 연도·추천 이유를 모두 포함했습니다. 다만 사용자가 명시적으로 공포 영화를 요청하지 않았음에도 '곡성'을 추가로 추천한 부분은 요청의 충실도에서 약간의 혼란을 줄 수 있어 1점 감점했습니다. 기본 추천인 '헤어질 결심'은 장르 설명에서 혼동의 여지가 있으나, 전체적으로는 요건을 충족합니다."}


지금까지 합성 데이터를 만들기 위한 기본 환경 설정과 프롬프트 엔지니어링, LLM as a Judge 방식을 배워보았습니다.

합성 데이터를 만드는 방식에 정답은 없습니다. 하지만, 가장 중요한 것은 real data와 가장 비슷하게 만들어내는 것입니다.

real data와 비슷하게 만든다는 것은 근본적으로 내가 만들고자 하는 서비스(만들고자 하는 모델)이 무엇이냐에 따라서 달라집니다.

따라서, 문제 정의(데이터의 목적)에 따라 다양하게 합성 데이터를 만들어보는 것을 시도해보시길 바랍니다.